## Demo 4 - Disentangling circular directions in toroidal dynamics

In [ ]:
# Imports

import numpy as np
from PIL import Image
import cv2
import glob

from sklearn.decomposition import PCA

from scipy import interpolate

from ripser import ripser
from persim import plot_diagrams

from dreimac import CircularCoords

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Auxiliary video functions

def getSlidingWindowVideo(I, dim, Tau, dT):
    N = I.shape[0] #Number of frames
    P = I.shape[1] #Number of pixels (possibly after PCA)
    pix = np.arange(P)
    NWindows = int(np.floor((N-dim*Tau)/dT))
    X = np.zeros((NWindows, dim*P))
    idx = np.arange(N)
    for i in range(NWindows):
        idxx = dT*i + Tau*np.arange(dim)
        start = int(np.floor(idxx[0]))
        end = int(np.ceil(idxx[-1]))
        f = interpolate.interp2d(pix, idx[start:end+1], I[idx[start:end+1], :], kind='linear')
        X[i, :] = f(pix, idxx).flatten()
    return X

def writeVideo(filename, frame_data, fps, resol ):
    n_row, n_col = resol
    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (n_col, n_row), False)
    for i in range(len(frame_data)):
        out.write(frame_data[i].reshape(n_row, n_col) )
    out.release()

def playVideo(filepath):
    cap = cv2.VideoCapture(filepath)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame', gray)
        if cv2.waitKey(30) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()   

def loadVideo(filepath):
    vid = cv2.VideoCapture(filepath)

    success,image = vid.read()
    count = 0
    success = True

    n_row, n_col, _ = image.shape

    data = []
    while success:
        data.append(image[...,0].reshape(-1)) 
        success,image = vid.read()
        count += 1

    vid.release() 
    data = np.array(data)

    print(count, " frames extracted")
    print("frame size = ", (n_row , n_col))
    print("data shape =", data.shape)
    
    return data , (n_row, n_col)

In [ ]:
data , shape = loadVideo('data/dots_quasi.wmv')
n_row, n_col = shape

In [ ]:
playVideo('data/dots_quasi.wmv')

In [ ]:
pca = PCA(n_components=25)
data_pca = pca.fit(data).transform(data)
plt.figure(figsize = (5,2))
plt.plot(pca.explained_variance_ratio_, '-*') ;
plt.title('Explained PCA variance - Original Data');

In [ ]:
res = ripser(data_pca, maxdim=2, n_perm =150, coeff =13)
dgms = res['dgms']
plt.figure(figsize = (3,3)) 
plot_diagrams(dgms)

---


<img src="https://i.ibb.co/3R9P6p7/SW.png" alt="test-1" border="0" width=650px>


In [ ]:
# Construct sliding window pointcloud of video data
wSize = 16
dim = 5
Tau = wSize/float(dim)

desiredSamples = 600
M = data.shape[0] - wSize + 1
dT = M/float(desiredSamples)

X = getSlidingWindowVideo(data, dim, Tau, dT)

XS = X - np.mean(X, 1)[:, None]
XS = XS/np.sqrt(np.sum(XS**2, 1))[:, None]

X = X - np.amin(X)
X = X/np.amax(X)
X = np.uint8(X*255)
frame_data = X[:, 0:data.shape[1]]

In [ ]:
# PCA of the Sliding window point cloud
pca = PCA(n_components=25)
XS_pca = pca.fit(XS).transform(XS)
plt.figure(figsize = (3.8,1.5))
plt.plot(pca.explained_variance_ratio_, '-*') ;
plt.title('Explained PCA variance -Sliding Window');

In [ ]:
# Compute persistence diagrams of the pca vesion of the sliding window point cloud
res = ripser(XS_pca, maxdim=2, n_perm =150, coeff =13, thresh = 0.9)
dgms = res['dgms']
plt.figure(figsize = (3,3)) 
plot_diagrams(dgms)

In [ ]:
# Fill in this cell : Compute the circular coordinates of the XS_pca data set

# Hint 1: cc = CircularCoords( , , , )
# Hint 2: help(CircularCoords)


In [ ]:
# Fill in this cell : Get the circular coordinates for two persistent cycles:

# Hint : theta1 = cc.get_coordinates( , )
#        theta2 = cc.get_coordinates( , )


In [ ]:
# Create a video with the frames ordered by theta1
sort_theta1 = np.argsort(theta1)
writeVideo('dots_theta1.mp4', frame_data[sort_theta1], 46 , (n_row , n_col) )

In [ ]:
# Play the video
# Want: one dot to move very quickly, while the other moves slowly
playVideo('dots_theta1.mp4')

In [ ]:
# Create a video with the frames ordered by theta2
sort_theta2 = np.argsort(theta2)
writeVideo('dots_theta2.mp4', frame_data[sort_theta2] , 46 , (n_row , n_col) )

In [ ]:
# Play the video
# Want: one dot to move very quickly, while the other moves slowly
playVideo('dots_theta2.mp4')